In [1]:
import pandas as pd
import numpy as np

import os
cwd = os.getcwd()
kaggle = cwd == "/kaggle/working"

# pretrain = pd.read_parquet("pretrain.parquet")
# train = pd.read_parquet("train.parquet")
# test = pd.read_parquet("test.parquet")

pretrain = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "pretrain.parquet")
train = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "train.parquet")
test = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "test.parquet")

#convert to string
train["text"] = train["text"].astype(str)
test["text"] = test["text"].astype(str)

#keep only the first 1000 rows
# pretrain = pretrain[:1000]
# train = train[:1000]
test = test[:2500]

In [2]:
import torch
from torch import nn
from transformers import TrainingArguments, Trainer, AutoTokenizer, XLMRobertaTokenizerFast, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup, TrainerCallback, TrainerControl, TrainingArguments
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=1)
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
config = model.config
tokenizer.model_max_length = config.max_position_embeddings
# model.classifier.out_proj = nn.Sequential(
#     nn.Linear(config.hidden_size, 1),
#     nn.Sigmoid()
# )

# tokenizer = AutoTokenizer.from_pretrained("microsoft/mdeberta-v3-base")
# model = AutoModelForSequenceClassification.from_pretrained("microsoft/mdeberta-v3-base", num_labels=1, ignore_mismatched_sizes=True)




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116_nocublaslt.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.6\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116_nocublaslt.dll...


c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\ProgramData\Anaconda3 did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Program Files/NVIDIA/CUDNN/v8.x'), WindowsPath('C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin'), WindowsPath('C:/Users/Timothe/.dotnet/tools'), WindowsPath('C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/libnvvp'), WindowsPath('C:/ProgramData/Anaconda3/Library/usr/bin'), WindowsPath('E:/texlive/2022/bin/win32')}
  warn(msg)
c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
Some weights of XLMRobertaF

In [3]:
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

for param in model.roberta.encoder.layer[-1].parameters():
    param.requires_grad = True
for param in model.roberta.encoder.layer[-2].parameters():
    param.requires_grad = True


# new_layers_to_add = 4

# from transformers.models.xlm_roberta.modeling_xlm_roberta import XLMRobertaLayer

# # Get the configuration of one existing layer to use as a template
# template_config = model.config

# # Manually create new layers using the template configuration
# new_layers = [XLMRobertaLayer(template_config) for _ in range(new_layers_to_add)]

# # Initialize the new layers with random weights
# for layer in new_layers:
#     layer.apply(model._init_weights)

# # Append new layers to the existing stack of layers
# model.roberta.encoder.layer.extend(new_layers)

# # Update the config to reflect the new number of layers
# model.config.num_hidden_layers += new_layers_to_add

# # Freeze the original layers
# for param in model.roberta.encoder.layer[:-new_layers_to_add].parameters():
#     param.requires_grad = False



In [4]:
from torch.utils.data import Dataset
import torch
import numpy as np

max_length = 128

def encode_texts(tokenizer, texts):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return torch.tensor(input_ids), torch.tensor(attention_masks)

pretrain_x, pretrain_attention_mask = encode_texts(tokenizer, pretrain['text'])
pretrain_y = torch.tensor(np.array(pretrain['label'].tolist()), dtype=torch.float32)

train_x, train_attention_mask = encode_texts(tokenizer, train['text'])
train_y = torch.tensor(np.array(train['label'].tolist()), dtype=torch.float32)

test_x, test_attention_mask = encode_texts(tokenizer, test['text'])
test_y = torch.tensor(np.array(test['label'].tolist()), dtype=torch.float32)



class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, label):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.label = label

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'label': self.label[idx],
        }

pretrain_dataset = CustomDataset(pretrain_x, pretrain_attention_mask, pretrain_y)
train_dataset = CustomDataset(train_x, train_attention_mask, train_y)
val_dataset = CustomDataset(test_x, test_attention_mask, test_y)

print(train_x.shape, train_attention_mask.shape, train_y.shape, test_x.shape, test_attention_mask.shape, test_y.shape, pretrain_x.shape, pretrain_attention_mask.shape, pretrain_y.shape)


torch.Size([1000, 128]) torch.Size([1000, 128]) torch.Size([1000]) torch.Size([1000, 128]) torch.Size([1000, 128]) torch.Size([1000]) torch.Size([1000, 128]) torch.Size([1000, 128]) torch.Size([1000])


In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F

model = model.cuda()
optimizer = AdamW(model.parameters(), lr=999)

# Training function
def train(model, lr_per_epoch, train_dataset, val_dataset):
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4)

    for epoch in range(len(lr_per_epoch)):
        model.train()
        lr = lr_per_epoch[epoch]
        optimizer.param_groups[0]['lr'] = lr
        train_loss = 0.0

        # Training loop with tqdm
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            inputs = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            labels = batch['label'].cuda()

            outputs = model(inputs, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            
            train_loss += loss.item()
            
        avg_train_loss = train_loss / len(train_loader)
        

        # Validation loop
        model.eval()
        val_loss = 0.0
        all_predictions_raw = []
        all_predictions = []
        all_labels = []

        # Validation loop with tqdm
        for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            inputs = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            labels = batch['label'].cuda()

            with torch.no_grad():
                outputs = model(inputs, attention_mask=attention_mask, labels=labels)
                logits = outputs.logits
                val_loss += outputs.loss.item()
                pred = F.sigmoid(logits)
                
                all_predictions_raw.append(pred.cpu())
                all_predictions.append(torch.round(pred).cpu())
                all_labels.append(labels.cpu())

        avg_val_loss = val_loss / len(val_loader)
        

        all_predictions_raw = torch.cat(all_predictions_raw)
        all_predictions = torch.cat(all_predictions)
        all_labels = torch.cat(all_labels)
        accuracy = accuracy_score(all_labels.numpy(), all_predictions.numpy())
        f1 = f1_score(all_labels.numpy(), all_predictions.numpy(), average='macro')

        print(f"Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}, Accuracy: {accuracy}, F1: {f1}")

        print(f"First predictions:")
        i = 0
        for y_pred, y in zip(all_predictions_raw, all_labels):
            print(f"y_pred: {y_pred.item()}, y: {y}")
            i += 1
            if i > 5:
                break

        print("\n")

# Train the model
# train(model, [1e-5, 1e-5, 1e-5], pretrain_dataset, val_dataset)
train(model, [1e-9, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5], train_dataset, val_dataset)

# Save the model
torch.save(model.state_dict(), '/kaggle/working/model.pt' if 'kaggle' in globals() else 'model.pt')


c:\ProgramData\Anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Validation Epoch 1: 100%|██████████| 250/250 [00:07<00:00, 34.24it/s]


Train Loss: 0.24551612832397224, Val Loss: 0.2147343755364418, Accuracy: 0.319, F1: 0.24184988627748297
First predictions:
y_pred: 0.5706850290298462, y: 0.0
y_pred: 0.5708164572715759, y: 0.0
y_pred: 0.5804279446601868, y: 1.0
y_pred: 0.5784091949462891, y: 1.0
y_pred: 0.5702053308486938, y: 0.0
y_pred: 0.5796269178390503, y: 1.0




Validation Epoch 2: 100%|██████████| 250/250 [00:07<00:00, 33.43it/s]


Train Loss: 0.2351092070043087, Val Loss: 0.1894984411597252, Accuracy: 0.32, F1: 0.24349965957484682
First predictions:
y_pred: 0.580420970916748, y: 0.0
y_pred: 0.5842334032058716, y: 0.0
y_pred: 0.6191871166229248, y: 1.0
y_pred: 0.6448922157287598, y: 1.0
y_pred: 0.565186083316803, y: 0.0
y_pred: 0.5924152135848999, y: 1.0




Validation Epoch 3: 100%|██████████| 250/250 [00:07<00:00, 32.47it/s]


...

Train Loss: 0.18926334313303234, Val Loss: 0.17550691456347703, Accuracy: 0.345, F1: 0.2844024870017994
First predictions:
y_pred: 0.5860961079597473, y: 0.0
y_pred: 0.5582557916641235, y: 0.0
y_pred: 0.6846325993537903, y: 1.0
y_pred: 0.6829226016998291, y: 1.0
y_pred: 0.5538344383239746, y: 0.0
y_pred: 0.6017970442771912, y: 1.0




Validation Epoch 5: 100%|██████████| 250/250 [00:07<00:00, 32.49it/s]


Train Loss: 0.1765116575062275, Val Loss: 0.16242653753701597, Accuracy: 0.452, F1: 0.4372306820189041
First predictions:
y_pred: 0.6013263463973999, y: 0.0
y_pred: 0.49808311462402344, y: 0.0
y_pred: 0.6444921493530273, y: 1.0
y_pred: 0.6954272985458374, y: 1.0
y_pred: 0.49415677785873413, y: 0.0
y_pred: 0.5850968360900879, y: 1.0




Training Epoch 6:  64%|██████▎   | 159/250 [00:06<00:04, 20.94it/s]

Training Epoch 6:  78%|███████▊  | 195/250 [00:08<00:02, 22.80it/s]